<a href="https://colab.research.google.com/github/awhitehouse1/DS4002-Project-2/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Pre-processing the data
df = pd.read_csv('/content/Final_time_series_data.csv')
print(df.info())
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108336 entries, 0 to 108335
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Year                    108336 non-null  int64  
 1   Month                   108336 non-null  int64  
 2   new_deaths_per_million  108335 non-null  float64
 3   Country                 108336 non-null  object 
 4   Date                    108336 non-null  object 
 5   GDP_per_Capita          107217 non-null  float64
 6   Democracy score         108336 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 5.8+ MB
None
   Year  Month  new_deaths_per_million      Country        Date  \
0  2020      1                     0.0  Afghanistan  2020-01-22   
1  2020      1                     0.0  Afghanistan  2020-01-23   
2  2020      1                     0.0  Afghanistan  2020-01-24   
3  2020      1                     0.0  Afghanistan  2020-01-25   
4 

In [ ]:

# Replace Inf and -Inf values with NaN
data = df.replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN values
data_cleaned = data.dropna()

# Check that there are no remaining NaN or Inf values
print(data_cleaned.isna().sum())  # Should print 0 for all columns

Year                      0
Month                     0
new_deaths_per_million    0
Country                   0
Date                      0
GDP_per_Capita            0
Democracy score           0
dtype: int64


In [ ]:
# Set the date column as the index for time series analysis
data_cleaned.set_index('Date', inplace=True)

# Inspect the final dataframe
print(data_cleaned.head())

            Year  Month  new_deaths_per_million      Country  GDP_per_Capita  \
Date                                                                           
2020-01-22  2020      1                     0.0  Afghanistan      512.055098   
2020-01-23  2020      1                     0.0  Afghanistan      512.055098   
2020-01-24  2020      1                     0.0  Afghanistan      512.055098   
2020-01-25  2020      1                     0.0  Afghanistan      512.055098   
2020-01-26  2020      1                     0.0  Afghanistan      512.055098   

            Democracy score  
Date                         
2020-01-22             2.85  
2020-01-23             2.85  
2020-01-24             2.85  
2020-01-25             2.85  
2020-01-26             2.85  


In [ ]:
# Vector AutoRegression (VAR) Model

In [ ]:
## Check for stationary. VAR model requires all the time series model to be stationary, meaning they have
## constant mean and variance.

In [ ]:
# Augmented Dickey-Fuller (ADF) Test
# Null Hypothesis (H0): The time series has a unit root, meaning it is non-stationary, and not suitable for
# VAR
# Alternative Hypothesis (H1): The time series does not have a unit root, meaning it is stationary. We can
# proceed with using this data for VAR model

In [ ]:
!pip install statsmodels

In [13]:

import pandas as pd
from statsmodels.tsa.stattools import adfuller

def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)
    for key,val in result[4].items():
        out[f'critical value ({key})']=val
    print(out.to_string())          # .to_string() removes the line "dtype: float64"

# check variables
# adf_test(data_cleaned['new_deaths_per_million'])
# adf_test(data_cleaned['GDP_per_Capita'])
adf_test(data_cleaned['Democracy score'])



Augmented Dickey-Fuller Test: 
ADF test statistic     -8.819805e+00
p-value                 1.890736e-14
# lags used             0.000000e+00
# observations          1.072150e+05
critical value (1%)    -3.430411e+00
critical value (5%)    -2.861567e+00
critical value (10%)   -2.566784e+00


In [ ]:
# The p value for the COVID 19 deaths is 0, which is less than the significance level of 0.05. Therefore, we
# reject the null hypothesis and can conclude that this variable is stationary.
# The p value for GDP per capita is also less than 0, so it is stationary. The p value for democracy score is
# also small, so all three variables are stationary. We can proceed with using this data for the VAR model.


In [18]:

from statsmodels.tsa.api import VAR

# Fit the VAR model with the cleaned data
model = VAR(data_cleaned)
results = model.fit(maxlags=15, ic='aic')

# Print model summary
print(results.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [23]:
## ??
data_cleaned['new_deaths_per_million'] = pd.to_numeric(data_cleaned['new_deaths_per_million'], errors='coerce')
data_cleaned['GDP_per_Capita'] = pd.to_numeric(data_cleaned['GDP_per_Capita'], errors='coerce')
data_cleaned['Democracy score'] = pd.to_numeric(data_cleaned['Democracy score'], errors='coerce')
print(data_cleaned.dtypes)

# Assuming data_cleaned is your DataFrame with relevant variables
model = VAR(data_cleaned)

# Use the select_order method to find the optimal lags
lag_order = model.select_order(maxlags=15)  # You can adjust maxlags as needed
print(lag_order.summary())


Year                        int64
Month                       int64
new_deaths_per_million    float64
Country                    object
GDP_per_Capita            float64
Democracy score           float64
dtype: object


<ipython-input-23-b7f13093b9a7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['new_deaths_per_million'] = pd.to_numeric(data_cleaned['new_deaths_per_million'], errors='coerce')
<ipython-input-23-b7f13093b9a7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['GDP_per_Capita'] = pd.to_numeric(data_cleaned['GDP_per_Capita'], errors='coerce')
<ipython-input-23-b7f13093b9a7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
# Correlation matrix of residuals shows the correlations between the residuals
# (errors) of the different variables
# new_deaths_per_million and GDP_per_Capita: 0.015304
# Very close to zero which suggests that the errors of these two variables are
# almost uncorrelated, meaning there is no strong linear relationship between the
# prediction errors of these variabls

# new_deaths_per_million and democracy score: 0.028133
# Also close to zero which suggests a weak or non-existent correlation between
# the errors of these two variables

# GDP_per_Capita and democracy score: 0.481471
# Moderate positive correlation which indicates that when the model makes errors
# in predicting GDP_per_Capita, those errors are moderately correlated with errors
# in predicting democracy score

# You want the residuals of different variables to be uncorrelated (close to 0)
# which indicates that the model is doing a good job of capturing the relationships
# between the variables, and the remaining prediction errors are random

# GDP_per_capita and democracy score suggest that the model might not be fully
# capturing the relationship between these variables

In [17]:
import pandas as pd
from statsmodels.tsa.api import VAR
# Panel VAR
# Ensure your DataFrame contains only the relevant columns
# For example, if your variables are ['new_deaths_per_million', 'GDP_per_Capita', 'Democracy_score']
data_for_var = data_cleaned[['new_deaths_per_million', 'GDP_per_Capita', 'Democracy score']]

# Fit the VAR model with the cleaned data
model = VAR(data_for_var)
results = model.fit(maxlags=15, ic='aic')

# Print model summary
print(results.summary())


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 23, Oct, 2024
Time:                     14:32:06
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    11.9446
Nobs:                     107202.    HQIC:                   11.9365
Log likelihood:      -1.09583e+06    FPE:                    152213.
AIC:                      11.9330    Det(Omega_mle):         152030.
--------------------------------------------------------------------
Results for equation new_deaths_per_million
                                coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------------------
const                             -0.037929         0.028978           -1.309           0.191
L1.new_deaths_per_million         -0.001336         0.003040           -0.440        

In [ ]:
# Correlation matrix of residuals shows the correlations between the residuals
# (errors) of the different variables
# new_deaths_per_million and GDP_per_Capita: _______


# new_deaths_per_million and democracy score: ______




In [ ]:
# Granger Casuality Test
from statsmodels.tsa.stattools import grangercausalitytests

# Specify the maxlag (based on your VAR model)
max_lag = 15

# Test Granger Causality for GDP per Capita on new_deaths_per_million
print("Granger Causality Test: GDP per Capita -> new_deaths_per_million")
grangercausalitytests(data_cleaned[['new_deaths_per_million', 'GDP_per_Capita']], max_lag)

# Test Granger Causality for Democracy score on new_deaths_per_million
print("\nGranger Causality Test: Democracy score -> new_deaths_per_million")
grangercausalitytests(data_cleaned[['new_deaths_per_million', 'Democracy score']], max_lag)

In [ ]:
# GDP per capita - New Deaths per million
# At 1-6 lags, the p-value is 0.0000, meaning that GDP per capita granger causes
# new deaths per million with strong statistical significance
# F-test and chi-squared values confirm that GDP per capita can help predict changes
# in new deaths per million during these time periods
# From 7+, the p-value becomes very high, indicating GDP per capita no longer
# granger causes new deaths per million

# Democracy score - New deaths per million
# At all lags 1-5, the p-values are 0.000, meaning democracy score strongly grander
# causes new deaths per million
# High F-test and chi-squared values indicate strong predictive power of democracy
# score on new deaths per million

In [ ]:
# SARIMAX Model
pip install statsmodels pandas matplotlib

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Select the target and exogenous variables
y = df['new_deaths_per_million']
exog = df[['GDP_per_Capita', 'Democracy score']]

# Define the SARIMAX model (replace order and seasonal_order with appropriate values)
model = SARIMAX(y, exog=exog, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))

# Fit the model
sarimax_model = model.fit(disp=False)

# Print model summary
print(sarimax_model.summary())


MissingDataError: exog contains inf or nans